## 1. Initialize settings

In [1]:
# change these to try this notebook out
BUCKET = 'cloudonair-ml-demo'
PROJECT = 'cloudonair-ml-demo'
REGION = 'us-central1'

In [2]:
import os

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash

gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## 2. Explore Natality dataset

In [4]:
# Create SQL query using natality data after the year 2000
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 100
"""

In [5]:
# Call BigQuery and examine in dataframe
import pandas
from pandas.io import gbq

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,3.562670,True,25,1,30
1,3.999185,False,30,1,32
2,7.438397,True,13,1,34
3,4.806077,True,19,1,34
4,4.812691,True,22,3,34


## 3. Training on Cloud ML Engine

In [6]:
%%bash

if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  # copy canonical set of preprocessed files if you didn't do previous notebook
  gsutil -m cp -R gs://cloud-training-demos/babyweight gs://${BUCKET}
fi

In [7]:
%%bash

gsutil ls gs://${BUCKET}/babyweight/preproc/*-00000*

gs://cloudonair-ml-demo/babyweight/preproc/eval.csv-00000-of-00012
gs://cloudonair-ml-demo/babyweight/preproc/train.csv-00000-of-00043


In [13]:
%%bash

OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBNAME=babyweight_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME

gsutil -m rm -rf $OUTDIR

gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/model_evaluation_pipeline/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.4 \
  -- \
  --bucket=${BUCKET} \
  --output_dir=${OUTDIR} \
  --train_examples=200000

gs://cloudonair-ml-demo/babyweight/trained_model us-central1 babyweight_180516_034734
jobId: babyweight_180516_034734
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [babyweight_180516_034734] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe babyweight_180516_034734

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs babyweight_180516_034734


## 3. Get a saved model directory

In [9]:
import datetime
import os
import pytz
import re
import urllib

from google.cloud import storage

def get_saved_model_dir(bucket, pattern='.*/babyweight/.*/saved_model.pb'):
  created_time = datetime.datetime(datetime.MINYEAR, 1, 1, tzinfo=pytz.timezone('UTC'))
  recent_model_path = None

  p = re.compile(pattern)
  
  for blob in storage.Client().bucket(bucket).list_blobs():
    if p.match(urllib.unquote(blob.path)):
      if created_time < blob.time_created:
        recent_model_path = blob.path
        created_time = blob.time_created

  saved_model = urllib.unquote(recent_model_path.split('/')[-1])
  saved_model_dir = '/'.join(saved_model.split('/')[:-1])
  return 'gs://{0}/{1}'.format(os.environ['BUCKET'], saved_model_dir)

saved_model_dir = get_saved_model_dir(os.environ['BUCKET'])

if saved_model_dir:
  os.environ['SAVED_MODEL_DIR'] = saved_model_dir
else:
  print('wait until saved_model.pb is written by Cloud ML Engine...')

## 4. Testing an evaluation pipeline with DirectRunner

### Configure a DirectRunner job

In [10]:
import os
import datetime

os.environ['RUNNER'] = 'DirectRunner'
os.environ['JOB_NAME'] = 'evaluate-ml-model-{0}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
os.environ['OUTPUT_TABLE'] = 'cloudonair-ml-demo:model_evaluation.sample'

### Execute a DirectRunner job

In [13]:
%%bash

cd $(pwd)/model_evaluation_pipeline/evaluator

python -m main \
  --datasize=100 \
  --year_from=1974 \
  --year_to=1978 \
  --saved_model_dir=${SAVED_MODEL_DIR} \
  --output_table=${OUTPUT_TABLE} \
  --project=${PROJECT} \
  --runner=DirectRunner \
  --region=${REGION} \
  --job_name=${JOB_NAME}

/usr/local/google/home/yaboo/Work/tf-estimator-tutorials/00_Miscellaneous/venv/lib/python2.7/site-packages/apache_beam/runners/direct/direct_runner.py:337: DeprecationWarning: options is deprecated since First stable release.. References to <pipeline>.options will not be supported
  pipeline.replace_all(_get_transform_overrides(pipeline.options))
INFO:root:Running pipeline with DirectRunner.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:initializing predictor...
2018-05-18 10:41:46.540789: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Writing 100 rows to cloudonair-ml-demo:model_evaluation.sample table.


### Show results of a DirectRunner Job

In [14]:
import pandas
from pandas.io import gbq

query = """
SELECT * FROM `{0}` LIMIT 10
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

,is_male,mother_age,plurality,gestation_weeks,weight_true,weight_predicted,weight_residual,model,testdata,time_inference
0,True,22,Single(1),39,7.312733,7.650989,-0.338255,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.811100
1,True,25,Single(1),39,6.499227,7.656961,-1.157734,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.864983
2,True,23,Single(1),39,5.937049,7.777191,-1.840142,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.840187
3,False,31,Single(1),33,4.186578,5.419439,-1.232861,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.910044
4,False,21,Single(1),42,8.999270,7.681918,1.317352,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.992060


In [15]:
import pandas
from pandas.io import gbq

query = """
SELECT
  model,
  testdata,
  SQRT(SUM(POW(weight_residual,2))/COUNT(*)) AS RMSE,
  AVG(time_inference) AS AVG_MSEC
FROM
  `{0}`
GROUP BY
  model, testdata
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

,model,testdata,RMSE,AVG_MSEC
0,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,1.031156,2.557452
1,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,1.055592,2.952533


## 5. Running an evaluation pipeline with DataflowRunner

### Configure a Dataflow job

In [115]:
import os
import datetime

os.environ['RUNNER'] = 'DataflowRunner'
os.environ['STAGING_LOCATION'] = 'gs://{0}/babyweight/staging'.format(os.environ['BUCKET'])
os.environ['TEMP_LOCATION'] = 'gs://{0}/babyweight/temp'.format(os.environ['BUCKET'])
os.environ['JOB_NAME'] = 'evaluate-ml-model-{0}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
os.environ['OUTPUT_TABLE'] = 'cloudonair-ml-demo:model_evaluation.prediction_results'

### Execute a Dataflow job

In [116]:
%%bash

cd $(pwd)/model_evaluation_pipeline/evaluator

python -m main \
  --datasize=1000000 \
  --year_from=1984 \
  --year_to=1988 \
  --saved_model_dir=${SAVED_MODEL_DIR} \
  --output_table=${OUTPUT_TABLE} \
  --project=${PROJECT} \
  --runner=${RUNNER} \
  --region=${REGION} \
  --staging_location=${STAGING_LOCATION} \
  --temp_location=${TEMP_LOCATION} \
  --job_name=${JOB_NAME} \
  --setup_file=$(pwd)/setup.py

running sdist
running egg_info
writing requirements to model_evaluation_demo.egg-info/requires.txt
writing model_evaluation_demo.egg-info/PKG-INFO
writing top-level names to model_evaluation_demo.egg-info/top_level.txt
writing dependency_links to model_evaluation_demo.egg-info/dependency_links.txt
reading manifest file 'model_evaluation_demo.egg-info/SOURCES.txt'
writing manifest file 'model_evaluation_demo.egg-info/SOURCES.txt'
running check
creating model-evaluation-demo-0.1
creating model-evaluation-demo-0.1/model_evaluation_demo.egg-info
creating model-evaluation-demo-0.1/process
copying files to model-evaluation-demo-0.1...
copying setup.py -> model-evaluation-demo-0.1
copying model_evaluation_demo.egg-info/PKG-INFO -> model-evaluation-demo-0.1/model_evaluation_demo.egg-info
copying model_evaluation_demo.egg-info/SOURCES.txt -> model-evaluation-demo-0.1/model_evaluation_demo.egg-info
copying model_evaluation_demo.egg-info/dependency_links.txt -> model-evaluation-demo-0.1/model_eva

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:32: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/__init__.py:95: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *
/usr/local/envs/py2env/lib/python2.7/site-packages/apache_beam/coders/typecoders.py:135: UserWarning: Using fallback coder for typehint: Dict[Any, Any].
  warnings.warn('Using fallback coder

In [18]:
import pandas
from pandas.io import gbq

query = """
SELECT * FROM `{0}` LIMIT 10
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

,is_male,mother_age,plurality,gestation_weeks,weight_true,weight_predicted,weight_residual,model,testdata,time_inference
0,True,16,Single(1),42,7.687519,7.750488,-0.062969,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,0.765085
1,True,17,Single(1),40,8.375361,7.844401,0.530960,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,0.846148
2,False,17,Single(1),41,7.063611,7.683310,-0.619699,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,0.854015
3,False,17,Single(1),99,7.438397,7.047079,0.391318,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,0.785828
4,False,19,Single(1),37,7.438397,6.930870,0.507527,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,0.883818


In [17]:
query = """
SELECT
  model,
  testdata,
  SQRT(SUM(POW(weight_residual,2))/COUNT(*)) AS RMSE,
  AVG(time_inference) AS AVG_MSEC
FROM
  `{0}`
GROUP BY
  model, testdata
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df

,model,testdata,RMSE,AVG_MSEC
0,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,1.031156,2.557452
1,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,1.055592,2.952533


## 6. Visualize Optimizing and Satisficing Metric with DataStudio

Now, you can create sharable dashboard using Google Data Studio like below. In the similar way, you can also create a dashboard for error analysis so that both business owners and engineers do simple error analysis.

<img src="model_evaluation_pipeline/dashboard1.png">